# SAE estimations for MPI in Cambodia

In [192]:
#Imports  
import pandas as pd
import geopandas as gpd
import os

In [ ]:
#Parameters
nis_year = 2006
dhs_year = 2005

#Load NIS information
root_path = '/Users/Daniel/Library/CloudStorage/OneDrive-VestigiumMétodosMixtosAplicadosSAS/proyectos/DISES/'

#This files contains the variable equivalence between the NIS and DHS datasets. 
common_variables_path = os.path.join(root_path, 'SAE/STATA DHS/common_variables.xlsx')
common_variables = pd.read_excel(common_variables_path)

### Load NIS data

In [195]:
nis_raw_data_path = f'SAE/STATA DHS/Stata NIS data/NIS_Village/NIS{nis_year}_Village_Year_Coords.xlsx'
nis_mpi_data_path = f'SAE/STATA DHS/Stata NIS data/MPI_NIS/MPI_NIS_{nis_year}.xlsx'

nis_path = os.path.join(root_path, nis_mpi_data_path)
nis_columns = common_variables['NIS_code'].to_list() #Only load the MPI related columns 
nis_columns = [x for x in nis_columns if not pd.isna(x)] #Drop nan to load the file properly
nis = pd.read_excel(nis_path)#, usecols=nis_columns)

In [196]:
#Read villages shape file
villages_path = os.path.join(root_path, 'SIG/DISES/Proximity/data/KHM-Villages/Villages.shp')
villages = gpd.read_file(villages_path)

In [197]:
nis = villages[['PHUMCODE','geometry']].merge(nis, left_on='PHUMCODE', right_on='VillGis', how='right')

In [198]:
nis = nis[['PHUMCODE', 	'geometry'] + nis_columns]

### Load DHS data

In [199]:
#Load DHS information
dhs_path = os.path.join(root_path, 'SAE/STATA DHS/stata_code/data/MPI/khm_dhs05_cot/khm_dhs05.csv')
dhs = pd.read_csv(dhs_path)

In [200]:
selected_dhs_variables = ['psu', 'strata', 'weight', 'd_cm', 'd_nutr',
       'd_satt', 'd_educ', 'd_elct', 'd_wtr', 'd_sani', 'd_hsg', 'd_ckfl',
       'd_asst', 'd_cm_01', 'd_nutr_01', 'd_satt_01', 'd_educ_01', 'd_elct_01',
       'd_wtr_01', 'd_sani_01', 'd_hsg_01', 'd_ckfl_01', 'd_asst_01']

In [201]:
#We use the average of the usehold level data to calculate the MPI variables
dhs = dhs[selected_dhs_variables].groupby('psu', as_index=False).mean()

In [202]:
#Attach cluster geometries to DHS cluster-level data (DHS)
dhs_shape_path = os.path.join(root_path, 'SAE/MPI/mpi_khm_dhs05_hmn_clust_CI_mis/mpi_khm_dhs05_hmn_clust_CI_mis.shp')
dhs_shape = gpd.read_file(dhs_shape_path)

In [203]:
import numpy as np

#Drop almos zero coordinates
# Define a small threshold for "almost zero"
epsilon = 1e-10

# Filter out points near (0, 0)
dhs_shape = dhs_shape[~(
    (np.abs(dhs_shape.geometry.x) < epsilon) &
    (np.abs(dhs_shape.geometry.y) < epsilon)
)]

In [204]:
#Keep only the rural clusters
dhs_shape = dhs_shape[dhs_shape['URBAN_RURA'] == 'R'] #Only rural clusters

In [205]:
dhs = dhs_shape[['DHSCLUST', 'geometry']].merge(dhs, right_on='psu', left_on='DHSCLUST', how='left')

In [206]:
#Import the dictionary with the common variables
common_variables

,Description,DHS_code,Description.1,NIS_code,DUMMY
0,Household had no child mortality under 18 in t...,d_cm,Child mortality rate calculated as the proport...,523-C_MORT_RATE,536-C_MORT_DUM
1,Household has no individuals malnourished,d_nutr,NaN,NaN,NaN
2,Household has all school age children up to 8 ...,d_satt,NaN,NaN,NaN
3,Household has at least one member with 6 years...,d_educ,Illiterate population rate calculated as the p...,513-ILLITERATE_RATE,537-ILLIT_DUM
4,Household has electricity,d_elct,Household with electricity rate calculated as ...,534-LQ_ROOF_ELEC_RATE,540-HOUSE_ELEC_DUM
5,"Household has roof, floor and walls that it is...",d_hsg,Low quality material roofs rate calculated as ...,NaN,NaN
6,Household has drinking water with MDG standards\n,d_wtr,"Rate of families with unimproved (pond, rivers...",517-UNIMPROV_WAT_RATE,538-UNIMPROV_WAT_DUM
7,Household has improved sanitation with MDG St...,d_sani,Household with latrines rate calculated as the...,532-Sanit_Rate,539-SANIT_DUM
8,Household has cooking fuel by MDG standards,d_ckfl,NaN,NaN,NaN
9,Household Asset Ownership: HH has car or more ...,d_asst,Rate of assets owneship (mobility & agricultur...,531-AGRI_ASSETS_RATE,NaN


In [207]:
nis = nis[nis.geometry.notnull()] #Exclude null geometries

In [208]:
dhs = dhs.to_crs(nis.crs)

In [209]:
dhs.to_file(f'data/mpi_underlying_nis/dhs-underlying-{dhs_year}.shp')

In [210]:
nis.to_file(f'data/mpi_underlying_nis/nis-complete-{nis_year}.shp')

/var/folders/wn/vzyjwy2x00q4stf523pr30jw0000gn/T/ipykernel_65478/3584320914.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  nis.to_file(f'data/mpi_underlying_nis/nis-complete-{nis_year}.shp')
/opt/anaconda3/envs/pymc_env/lib/python3.13/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: '523-C_MORT_RATE' to '523-C_MORT'
  ogr_write(
/opt/anaconda3/envs/pymc_env/lib/python3.13/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: '513-ILLITERATE_RATE' to '513-ILLITE'
  ogr_write(
/opt/anaconda3/envs/pymc_env/lib/python3.13/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: '534-LQ_ROOF_ELEC_RATE' to '534-LQ_ROO'
  ogr_write(
/opt/anaconda3/envs/pymc_env/lib/python3.13/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: '517-UNIMPROV_WAT_RATE' to '517-UNIMPR'
  ogr_write(
/opt/anaconda3/envs/pymc_env/lib/pyt